# Setup

In [2]:
# importing
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 

In [3]:
# loading the data
df = pd.read_csv('churn_bank.csv')

NRows = df.shape[0]
NColumns = df.shape[1]

columnNames = list(df.columns)

In [4]:
# Creating two additional datasets for comparison
dfStayed = df.loc[df['Exited']==1]
dfExited = df.loc[df['Exited']==0]

# General Analysis

In [16]:
#pip install tabulate
from tabulate import tabulate

In [82]:
dfExited.describe().T

,count,mean,std,min,25%,50%,75%,max
CustomerId,7963.0,1.569117e+07,71744.234941,15565701.00,15628818.50,15691543.00,1.575335e+07,15815690.00
CreditScore,7963.0,6.518532e+02,95.653837,405.00,585.00,653.00,7.180000e+02,850.00
Age,7963.0,3.740839e+01,10.125363,18.00,31.00,36.00,4.100000e+01,92.00
Tenure,7963.0,5.033279e+00,2.880658,0.00,3.00,5.00,7.000000e+00,10.00
Balance,7963.0,7.274530e+04,62848.040701,0.00,0.00,92072.68,1.264103e+05,221532.80
NumOfProducts,7963.0,1.544267e+00,0.509536,1.00,1.00,2.00,2.000000e+00,3.00
HasCrCard,7963.0,7.071455e-01,0.455101,0.00,0.00,1.00,1.000000e+00,1.00
IsActiveMember,7963.0,5.545649e-01,0.497045,0.00,0.00,1.00,1.000000e+00,1.00
EstimatedSalary,7963.0,9.973839e+04,57405.586966,90.07,50783.49,99645.04,1.486100e+05,199992.48
Exited,7963.0,0.000000e+00,0.000000,0.00,0.00,0.00,0.000000e+00,0.00


In [108]:
dfStayed.describe().T

,count,mean,std,min,25%,50%,75%,max
CustomerId,2037.0,1.569005e+07,72692.623920,15565706.00,15627360.00,15688963.00,15753092.00,15815656.00
CreditScore,2037.0,6.453515e+02,100.321503,350.00,578.00,646.00,716.00,850.00
Age,2037.0,4.483800e+01,9.761562,18.00,38.00,45.00,51.00,84.00
Tenure,2037.0,4.932744e+00,2.936106,0.00,2.00,5.00,8.00,10.00
Balance,2037.0,9.110854e+04,58360.794816,0.00,38340.02,109349.29,131433.33,250898.09
NumOfProducts,2037.0,1.475209e+00,0.801521,1.00,1.00,1.00,2.00,4.00
HasCrCard,2037.0,6.990673e-01,0.458776,0.00,0.00,1.00,1.00,1.00
IsActiveMember,2037.0,3.608247e-01,0.480358,0.00,0.00,0.00,1.00,1.00
EstimatedSalary,2037.0,1.014657e+05,57912.418071,11.58,51907.72,102460.84,152422.91,199808.10
Exited,2037.0,1.000000e+00,0.000000,1.00,1.00,1.00,1.00,1.00


In [128]:
## calculating the differences in summary statistics
columnNamesDescribe = list(df.columns.drop(['Surname', 'Geography', 'Gender']))
columns2keep = ['CreditScore','Age','Tenure','Balance','EstimatedSalary']
columns2Drop = [val for val in columnNamesDescribe if val not in columns2keep]

differences_in_trends = np.array(dfExited.describe().T.drop(columns2Drop)) - np.array(dfStayed.describe().T.drop(columns2Drop))
#differences_in_trends = (np.array(dfExited.describe().T.drop(columns2Drop)) - np.array(dfStayed.describe().T.drop(columns2Drop)))/np.array(dfStayed.describe().T.drop(columns2Drop))
#len(columnNames)-len(differences_in_trends)
differences_in_trends = pd.DataFrame(differences_in_trends).set_index(pd.Index(columns2keep))

headers=['variable', 'count','mean', 'std', 'min', '25%', '50%', '75%', 'max']
table = [pd.DataFrame(columns2keep), pd.DataFrame(differences_in_trends)]
print(tabulate(differences_in_trends, headers))

variable           count           mean           std    min        25%       50%       75%        max
---------------  -------  -------------  ------------  -----  ---------  --------  --------  ---------
CreditScore         5926       6.5017      -4.66767    55          7          7        2          0
Age                 5926      -7.42961      0.363801    0         -7         -9      -10          8
Tenure              5926       0.100535    -0.0554485   0          1          0       -1          0
Balance             5926  -18363.2       4487.25        0     -38340     -17276.6  -5023.05  -29365.3
EstimatedSalary     5926   -1727.29      -506.831      78.49   -1124.23   -2815.8  -3812.95     184.38


* Credit Score: no big difference between exited and stayed, except that it seems the min value of those who exited is higher.
* Age: those who exit are on average 7 years younger.
* tenure: Those who exit seem to have been clients of the bank for a bit longer, but not a significative amount of time in average.
* Balance: Those who exit seem to have a smaller average balance (of almost 19000).
* EstmatedSalary: those who exit have a slightly smaller salary but nothing huge/significative.